# 시계열 분석
* 시계열 데이터 : 주가, 환율, 거래량, 판매량, 재고량, 수요량, 클릭률, 기온, 습도, 인구, 출생률, 트래픽양의 피크시간 패턴, 제품의 판매주기, 심장 박동


* 시계열 데이터의 특성을 파악
    * 규칙적 : 경향(trend), 계절성(seasonality), 주기(cycle)
    * 불규칙적 : 불규칙성(irregular, random)

#### 시계열의 구성요소
  * 추세 - 추세에는 장기간에 걸친 시계열 데이터의 일반적인 방향이 표시됩니다. 추세는 증가(상향), 감소(하향) 또는 수평(정지)일 수 있습니다.
  * 계절성 - 계절성 성분은 시기, 방향 및 크기와 관련하여 반복되는 추세를 나타냅니다. 더운 날씨로 인한 여름철 물 소비량 증가가 대표적입니다.
  * 순환 성분 - 특정 기간 동안 설정된 반복이 없는 추세입니다. 사이클(cycle)은 주로 경기 순환에서 관찰되는 시계열의 흥망성쇠, 호황, 슬럼화 시기를 가리킵니다. 이러한 주기는 계절적 변화를 나타내지는 않지만, 일반적으로 시계열의 특성에 따라 3~12년에 걸쳐 일어납니다.
 * 불규칙 변동 - 추세 및 주기적 변동을 제거할 때 분명하게 나타나는 시계열 데이터의 변동입니다. 이러한 변화는 예측할 수 없고 불규칙하며 무작위일 수도 있고 아닐 수도 있습니다.
  * ETS(Error, Trend, Seasonality) 분해 - ETS 분해는 시계열의 여러 성분을 구분하는 데 사용됩니다.

#### 응용

* 추세 파악 
: 무작위 적인 소음을 제거하여 흐름을 파악
* 원인 예측 및 대응
: 수요 분석을 통한 재고량 관리
* 전망
: 영업 전략, 생산 계획



## 통계청 서울특별시 아파트 거래 현황
* 부동산거래현황:행정구역별 아파트거래현황 : https://kosis.kr/statHtml/statHtml.do?orgId=408&tblId=DT_408_2006_S0049&conn_path=I2

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
from IPython.display import set_matplotlib_formats

import plotly.io as pio
pio.renderers.default = "colab"

def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux(colab)
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family

plt.rc('font', family=get_font_family()) 
plt.rc('axes', unicode_minus=False)

set_matplotlib_formats('retina')
%matplotlib inline

## csv 파일로드

In [ ]:
df = pd.read_csv("kosis-apt-2006-2021.csv", encoding="cp949")
df.shape

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

In [ ]:
df.info()

In [ ]:
df["연도"] = df["시점"].apply(lambda x : int(x[:4]))
df["월"] = df["시점"].apply(lambda x : int(x[5:]))
df

In [ ]:
df = df.rename(columns={"호수 (호수)": "거래수"})

In [ ]:
df = df[["시점", "연도", "월", "거래수"]]
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(15, 5))
sns.barplot(data=df, x="시점", y="거래수")

In [ ]:
df.set_index("시점")["거래수"].plot(figsize=(12, 5))

In [ ]:
df.set_index("시점")["거래수"].plot.bar(figsize=(12, 5))

In [ ]:
plt.figure(figsize=(15, 5))
sns.lineplot(data=df, x="연도", y="거래수")

In [ ]:
plt.figure(figsize=(15, 5))
sns.pointplot(data=df, x="연도", y="거래수")

In [ ]:
plt.figure(figsize=(15, 5))
sns.pointplot(data=df, x="월", y="거래수")

In [ ]:
plt.figure(figsize=(15, 5))
sns.pointplot(data=df, x="월", y="거래수", estimator=np.sum, ci=None)

In [ ]:
# 수치 데이터를 히스토그램으로 표현
# 수치 데이터를 막대그래프로 표현하기 위해서는 도수분포표를 만들고 이를 시각화 하는 것이 히스토그램
df.hist(figsize = (15,15), bins=25)

## plotly

In [ ]:
fig = px.line(df, x="시점", y="거래수")
fig.show(renderer="colab")

In [ ]:
fig = px.bar(df, x="시점", y="거래수")
fig.show(renderer="colab")



## 이동평균
통계학에서 이동평균(롤링평균 또는 이동평균)은 전체 데이터 집합의 여러 하위 집합에 대한 일련의 평균을 만들어 데이터 요소를 분석하는 계산이다. 이동산술평균 (Moving Mean) 또는 롤링산술평균(Rolling Mean)이라고도 한다.

일련의 연속된 숫자와 고정된 부분 집합 크기가 주어지면, 이동 평균의 첫 번째 요소는 연속된 숫자의 첫 고정 부분 집합의 평균을 취하여 구한다. 그런 다음 "앞으로 이동"하여 하위 집합을 변경한다. 즉, 부분 집합의 첫 번째 숫자를 제외하고 연속된 숫자의 다음 값을 포함시킨다.


* 출처 : [이동평균 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%9D%B4%EB%8F%99%ED%8F%89%EA%B7%A0)

* 이동평균의 종류
    * 단순이동평균(Simple Moving Average)
    * 누적이동평균(Cumulative moving average)
    * 가중이동평균(Weighted Moving Average)
    * 지수이동평균(Exponential Moving Average)


### rolling
* window : 몇 개의 데이터를 가지고 연산을 할 것인지 결정
    * 의미 : window가 클수록 부드러운 선의 형태를 가지고 있음 window가 center에 있지 않으면 window는 클수록 위상 차가 큼
* min_periods : 최소한 몇 개의 데이터가 있어야 연산을 할 것인지 결정
    * --> window 안의 데이터가 min_periods보다 적다면, 
       결과는 NaN
* center : window의 위치를 window 끝으로 할 것인지 중간으로 할 것 인지 결정
    * 의미 : 시계열 데이터에서는 window를 중간으로 할 수 없다(미래의 데이터는 볼 수 없기 때문에)
* <font color="red">연산을 mean()으로 했을 경우 'rolling'은 전체적으로 진짜 데이터에서 **잡음(noise)을 제거**하는 의미</font>

### expanding

* window : expanding에서는 window를 결정하지 않음
    * 왜냐하면 window의 크기가 순차적으로 증가하기 때문
* min_periods : 최소한 몇 개의 데이터가 있어야 연산을 할 것인지 결정
    * --> 이것은 rolling과 같음
* expanding에서의 sum()은 cumsum() 연산과 동일하게 진행되나 NaN 이 있을 경우 expanding은 무난하게 처리할 수 있음
* 연산을 sum()으로 했을 경우, cumsum()과 같은 의미가 있음
* <font color="red">연산을 mean()으로 했을 경우 'expanding' 은 전체적으로 **추세**를 찾는 의미가 있음</font>

<img src="https://i.imgur.com/mCBaCbd.png">



### 단순이동평균(SMA: Simple Moving Average)
* rolling을 사용합니다.

In [ ]:
df["시점"] = pd.to_datetime(df["시점"])

In [ ]:
df_ym = df.set_index("시점")["거래수"]
df_ym.head()

In [ ]:
# rolling mean을 구해서 시각화 합니다.
rolmean = df_ym.rolling(window=12).mean() 
rolstd = df_ym.rolling(window=12).std()

In [ ]:
df_ym.plot(label='Original')
rolmean.plot(label='Rolling Mean')
rolstd.plot(label='Rolling Std', figsize=(15, 5))

plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')

## Weighted Moving Average
* exponentially-weighted-windows
* https://pandas.pydata.org/pandas-docs/stable/user_guide/window.html#exponentially-weighted-window

In [ ]:
df_ym = df_ym.to_frame()
df_ym['WMA12'] = df_ym['거래수'].ewm(span=12).mean()
df_ym['WMA6'] = df_ym['거래수'].ewm(span=6).mean()
df_ym

In [ ]:
df_ym[['거래수', 'WMA6','WMA12']].plot(figsize=(15,5))

## Simple Exponential Smoothing
### 지수이동평균(Exponential Moving Average)
* 최근 자료에 가중치를 부여하여 최근 가격 변동을 잘 나타냅니다.

In [ ]:
split_count = int(df_ym.shape[0] * 0.8)
split_count

In [ ]:
# 데이터셋을 train과 test 로 나눈다.
train = pd.DataFrame(df_ym[:split_count])
test = pd.DataFrame(df_ym[split_count:])

In [ ]:
train["거래수"].plot(figsize=(15,5))
test["거래수"].plot(title='train과 test세트로 분할')

In [ ]:
from statsmodels.tsa.api import SimpleExpSmoothing
ses_model = SimpleExpSmoothing(np.asarray(train['거래수'].astype(np.float)))

In [ ]:
ses_result = ses_model.fit()
ses_result

In [ ]:
y_hat = test.copy()
y_hat['SES'] = ses_result.forecast(len(test))
y_hat['SES'][:3]

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(train['거래수'], label='Train')
plt.plot(test['거래수'], label='Test')
plt.plot(y_hat['SES'], label='Simple Exp Smoothing')
plt.legend()

In [ ]:
rmse = {}
# root mean squared error 로 오차를 계산해 본다.
rmse["SES"] = np.sqrt(np.square(test['거래수'] - y_hat['SES']).mean())
rmse["SES"]

## Expanding

### 누적이동평균(Cumulative moving average)
* expanding을 사용합니다. window 창이 시간의 흐름에 따라 증가합니다.

In [ ]:
# Expanding
df_ym['거래수'].expanding(min_periods=1).mean().plot(figsize=(15,5))

## Dickey–Fuller test

In [ ]:
# 결과의 p-value 가 5%를 벗어나면 non-stationary 데이터
# AIC - 회귀에서 예측변수(predictor)를 고를 때 사용하며, 아카이케(Akaike)의 정보 기준(AIC; Akaike’s information Criterion)
# AIC 출처 : [8.6 추정과 차수 선택 | Forecasting: Principles and Practice](https://otexts.com/fppkr/arima-estimation.html)
from statsmodels.tsa.stattools import adfuller
# Perform Dickey–Fuller test:
print('Results of Dickey Fuller Test:')
dftest = adfuller(df_ym['거래수'], autolag='AIC')
dfoutput = pd.Series(
    dftest[0:4], 
    index=['Test Statistic',
           'p-value',
           '#Lags Used',
           'Number of Observations Used'])

for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

## Time Series data Decomposition(시계열 데이터 분해)
* Seasonal
* Trend
* Residual(random, remainder)




### 정상 시계열(stationary)
 * 뚜렷한 추세가 없음
 * 진폭이 흐름에 따라 일정함

### 비정상 시계열(non-stationary)
 * 평균이 시간대에 따라 다름
 * 추세, 계절성을 가짐
 * 분산이 변한다
 * 분산이 일정하지 않으면 특정 기간에 오류가 발생하고 예측을 하기에 적합하지 않을 수 있다.


### 비정상 시계열의 정상화


1. 분산이 일정하지 않은 경우 
 * 분산 안정화 변환(로그변환, 제곱근 변환, Box-Cox 변환)
2. 추세가 있을 때
 * 결정적 추세 : 분해법 또는 추세항 모형에 포함
 * 확률적 추세(Dickey-Fuller의 단위근 검정): 차분을 이용 Trend 제거
 * 로그변환(lograithm) : 표준편차가 자료의 크기에 비례 해서 증가 할 때
3. 계절성을 가지는 경우
 * 결정적 계절추세 : 계절 추세항 모형에 포함
 * 확률적 계절추세(계절형 단위근 검정) : 계절차분 
 

## Additive Model

* 값을 더해서 구한다.

$y_t = Level + Trend + Seasonality + Noise$


In [ ]:
from statsmodels.api import tsa

# Additive model
res = tsa.seasonal_decompose(df_ym["거래수"], freq=30, model="additive")
fig = res.plot()

## Multiplicative Model
*  값을 곱해서 구한다.

$y_t = Level \times Trend \times Seasonality \times Noise$

In [ ]:
# multiplicative
res = tsa.seasonal_decompose(df_ym["거래수"], freq=7, model="multiplicative")
fig = res.plot()

## ACF, PACF
* ARIMA 모델 사용시 lag 값에 따른 절단값으로 p,d,q값을 찾기 위해 그려봅니다.
* AUTO.ARIMA 함수를 사용하여 최적화된 파라미터를 찾을 수도 있습니다.

## ACF : 자기상관 함수 AutoCorrelation Function

## PACF : 부분 자기상관 함수 Partial AutoCorrelation Function 

* p – Lag value where the PACF chart crosses the upper confidence interval for the first time.
* q – Lag value where the ACF chart crosses the upper confidence interval for the first time.

* 참고 : [Detecting stationarity in time series data - Towards Data Science](https://towardsdatascience.com/detecting-stationarity-in-time-series-data-d29e0a21e638)

In [ ]:
from statsmodels.graphics import tsaplots
# lag는 0부터 설정할 수 있으나 너무 낮으면  그래프를 보기 어려움
# 0은 자기 자신이라 제외하고 봄

ax1 = plt.subplot(211)
tsaplots.plot_acf(df_ym["거래수"], lags=30, ax=ax1)
ax2 = plt.subplot(212)
tsaplots.plot_pacf(df_ym["거래수"], lags=30, ax=ax2)
plt.tight_layout()

## ARIMA

* 자동회귀이동평균(ARMA : Autoregressive moving average) 모델의 일반화
* 시계열 데이터를 더 잘 이해하거나 미래 지점을 예상(예측)하기에 적합
* 데이터가 비정상성이 아닌 증거를 나타내는 경우에 적용되며, **초기 차분 단계(모델의 "통합된" 부분에 해당)를 한 번 이상 적용하여 비정상성을 제거** >> 비정상데이터도 다룰 수 있음

* AR 부분은 진화하는 관심 변수가 시차(즉, 이전) 값으로 회귀됨을 나타냄
* MA 부분은 회귀 오류가 실제로 과거 여러 시간에 동시에 발생한 오류 항의 선형 조합임을 나타냄
* I("누적"의 경우)는 데이터 값이 해당 값과 이전 값의 차이로 바뀌 었음을 나타냄

<img src = "https://imgur.com/5sV0YS5.jpg">

## Seasonal ARIMA
* 참고 : [An End-to-End Project on Time Series Analysis and Forecasting with Python](https://towardsdatascience.com/an-end-to-end-project-on-time-series-analysis-and-forecasting-with-python-4835e6bf050b)

ARIMA(p,d,q)모형은 차수 p, d, q의 값에 따라 다른이름으로 불립니다.

- 자동 회귀(p) -> AR모형과 관련
      ex) p=0이면, IMA(d,q)모형이라 부르고 d번 차분하면 MA(q)모형을 따르게 됨
- 적분(d) -> 정상성에 필요한 비계절차의 수
      ex) d=0이면, ARMA(p,q)모형이라 부르고 이 모형은 정상성을 만족한다.
- 이동 평균(q) -> 예측 방정식의 지연된 예측 오류 수, MA모형과 관련이 있음
      ex) q=0이면, ARI(p,d)모형이라 부르며, d번 차분한 시계열이 AR(p)모형을 따르게 됨


### Trend
* p: Trend autoregression order. AR(p)모형의 p차수
* d: Trend difference order. 트랜드를 제거하여 안정시계열로 만들기 위한 I(d)의 차분 차수 d
* q: Trend moving average order. MA(q)의 q차수 

### Seasonal
* P: Seasonal autoregressive order.
* D: Seasonal difference order.
* Q: Seasonal moving average order.
* m: The number of time steps for a single seasonal period.


In [ ]:
import itertools
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
m = 12
print(pdq)
seasonal_pdq = [(x[0], x[1], x[2], m) for x in list(itertools.product(p, d, q))]
seasonal_pdq

In [ ]:
# Grid Search와 유사하게 최적의 파라메터 값을 찾는다.
y = train['거래수'].to_list()

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = tsa.statespace.SARIMAX(y,
                                        order=param,
                                        seasonal_order=param_seasonal,
                                        enforce_stationarity=False,
                                        enforce_invertibility=False)
            results = mod.fit()
            print(f"ARIMA{param}x{param_seasonal}{m} - AIC:{results.aic}")
        except:
            continue

In [ ]:
# ARIMA(p,d,q)(P,D,Q)m
arima = tsa.statespace.SARIMAX(train['거래수'].to_list(),
                                  order=(1,1,1),
                                  seasonal_order=(0,1,1,12),
                                  enforce_stationarity=False,
                                  enforce_invertibility=False)
# 학습
arima_result = arima.fit()
arima_result.summary()

## 잔차 검정
잔차가 정상성을 만족(백색소음 과정인지)하는지, 정규성 및 등분산성을 만족하는지 파악합니다.
1. summary 결과에서 Ljung-Box(Q) / Heteroskedasticity(H) / Jarque-Bera(JB) 검정 만족 여부를 파악할 수 있습니다.
2. plot_diagnostics 잔차 그래프로도 정상성과 정규성을 만족하는지 파악할 수 있습니다.

* Ljung-Box(Q) / Heteroskedasticity(H) / Jarque-Bera(JB)에 대한 부분은 모두 잔차에 대한 검정 통계량입니다.
  * Ljung-Box(Q)(융-박스 검정 통계량) - 잔차가 백색소음인지 검정한 통계량으로 Prob(Q) 값을 보면 0.79이므로 유의수준 0.05에서 귀무가설을 기각하지 못합니다. Ljung-Box (Q) 통계량의 귀무가설은 “잔차(residual)가 백색소음(white noise) 시계열을 따른다”이므로, 위 결과를 통해 시계열 모형이 잘 적합되었고 남은 잔차는 더이상 자기상관을 가지지 않는 백색소음임을 확인할 수 있습니다.

* Jarque-Bera(JB)(자크-베라 검정 통계량)은 잔차가 정규성을 띠는지 검정한 통계량으로, Prob(JB)값을 보면 0.03으로 유의 수준 0.05에서 귀무가설을 기각합니다. Jarque-Bera (JB) 통계량의 귀무가설은 “잔차가 정규성을 만족한다”이므로, 위 결과를 통해 “잔차가 정규성을 따르지 않음”을 확인할 수 있습니다.

* Heteroskedasticity(H)(이분산성 검정 통계량)은 잔차가 이분산을 띠지 않는지 검정한 통계량입니다.

### 왜도(Skewness)
  * 비대칭도(非對稱度, skewness) 또는 왜도(歪度)는 실수 값 확률 변수의 **확률 분포 비대칭성을 나타내는 지표** >> 데이터의 분포가 정규분포에 비해서 얼마나 비대칭인지 나타내는 척도
<img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/c/cc/Relationship_between_mean_and_median_under_different_skewness.png/1024px-Relationship_between_mean_and_median_under_different_skewness.png">

  * 음의 왜도(Negative Skewness): 데이터의 중심(평균)이 정규 분포보다 오른쪽으로 치우쳐져 있고, 꼬리는 왼쪽으로 길게 표현
  * 양의 왜도(Positive Skewness): 데이터의 중심(평균)이 정규 분포보다 왼쪽으로 치우쳐져 있고, 꼬리는 오른쪽으로 길게 표현
  * 왜도의 값이 -2 미만이면 Negative Skewness, 2 초과이면 Positive Skewness
    * 대칭적 데이터: 왜도가 -0.5와 0.5사이
    * 적당히 치우친 데이터: 왜도가 -1과 -0.5(음의 치우침) 또는 0.5와 1(양의 치우침) 사이의 값
    * 고도로 치우친 데이터: 왜도가 -1보다 작거나(음으로 치우침) 1보다 큰 값(양으로 치우침)

* 참조: https://en.wikipedia.org/wiki/Skewness


In [ ]:
arima_result.summary().tables[1]

### 잔차 검정
* `ARIMAResults.plot_diagnostics(variable=0, lags=10, fig=None, figsize=None, truncate_endog_names=24, auto_ylims=False, bartlett_confint=False, acf_kwargs=None)`

|매개변수|설명|기본값|
|-----|-----|:-----:|
|variable|진단 그림을 만들어야 하는 내인성 변수의 인덱스|0|
|lags|상관 도표에 포함할 시차 수|10|
|fig|지정된 경우 하위 플롯은 새 그림 대신 이 그림에 작성<br> 2x2 그리드는 fig.add_subplot()를 사용하여 제공된 그림에 생성||
|figsize|그림이 작성되면 이 인수를 통해 크기를 지정 >> (너비, 높이)||
|auto_ylims|True이면 Y축 한계를 ACF 값으로 자동 조정||
|bartlett_confint|ACF 값에 대한 신뢰 구간은 일반적으로 r_k 주위에 2개의 표준 오차가 있음<br>표준 오차에 사용되는 공식은 상황에 따라 다름<br>자기 상관을 사용하여 ARIMA 과정의 일부로 잔차의 랜덤성을 검정하는 경우 잔차가 백색소음이라고 가정하여 표준 오차가 결정||
|acf_kwargs|flot_acf()에서 생성된 상관 관계 Matplotlib 플롯에 직접 전달되는 키워드 인수의 선택적 사전||



* Standardized residual, Correlogram - 잔차가 백색소음을 따르는지 보여줌
  - Standardized residual : 잔차를 시계열로 그린 것
  - Correlogram은 잔차에 대한 ACF

* Histogram plus estimated density, Normal Q-Q - 잔차가 정규성을 만족하는지 보여줌
  - Histogram plus estimated density는 잔차의 히스토그램을 그려 정규 분포 N(0,1)과 밀도를 추정한 그래프를 같이 겹쳐서 보여줌
  - Normal Q-Q그래프는 Q-Q 플랏으로 정규성을 만족한다면 빨간 일직선 위에 점들이 분포


In [ ]:
arima_result.plot_diagnostics(figsize=(15, 10))

In [ ]:
# 예측 세트 갯수
test.shape

In [ ]:
# 예측 세트 인덱스 시작, 끝 번호 구하기
start_no = split_count
end_no = split_count + test.shape[0] - 1
start_no, end_no

In [ ]:
# 예측
predict_value = arima_result.predict(start=start_no, end=end_no, dynamic=True)
predict_value[:5]

In [ ]:
y_hat['ARIMA'] = predict_value
y_hat['ARIMA'].head()

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(train['거래수'], label='Train')
plt.plot(test['거래수'], label='Test')
plt.plot(y_hat['ARIMA'], label='Seasonal ARIMA')
plt.legend()

In [ ]:
# root mean squared error 로 오차를 계산해 본다.
rmse["ARIMA"] = np.sqrt(np.square(test['거래수'] - y_hat['ARIMA']).mean())
rmse["ARIMA"]

In [ ]:
rmse

## fbprophet
* 공식문서 : https://facebook.github.io/prophet/docs/quick_start.html#python-api
* https://anaconda.org/conda-forge/fbprophet
* `conda install -c conda-forge fbprophet`
* pip로 설치한다면 pystan이 설치되어 있어야 함

In [ ]:
# !pip install pystan
# !pip install fbprophet

In [ ]:
from fbprophet import Prophet

In [ ]:
df_p = df[["시점", "거래수"]].copy()
df_p.columns = ["ds", "y"]
df_p["y"] = np.log(df_p["y"])
df_p

In [ ]:
p_train = df_p[:split_count]
p_test = df_p[split_count:]
p_train.shape, p_test.shape

### Prophet에서 조정이 가능한 파라미터

- 데이터의 상한 하한의 설정
- 변동 포인트의 설정
- 계절성의 조정
- 특별 이벤트의 추가
- 신뢰 구간의 조정

In [ ]:
m = Prophet(seasonality_mode='multiplicative')
m.fit(p_train)

In [ ]:
test.index

In [ ]:
test.shape[0]

In [ ]:
# make_future_dataframe 로 미래 데이터를 만들 수도 있으나 여기에서는 미리 나누어 준 p_test 변수를 사용합니다.
# future = m.make_future_dataframe(periods=1, freq="MS")
# future.tail()

In [ ]:
forecast = m.predict(p_test)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
df_forecast = forecast[['ds', 'yhat']]
df_plt_forecast = df_forecast.set_index('ds')
df_plt_forecast.shape

In [ ]:
df_plt_forecast

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(np.log(train['거래수']), label='Train')
plt.plot(np.log(test['거래수']), label='Test')
plt.plot(df_plt_forecast['yhat'], label='fbprophet')
plt.legend()
plt.title("로그 변환 예측값")

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(train['거래수'], label='Train')
plt.plot(test['거래수'], label='Test')
plt.plot(np.exp(df_plt_forecast['yhat']), label='fbprophet')
plt.legend()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
# plotly 가 설치되어 있지 않다면 아래의 명령어로 설치가 필요
# 아나콘다에 설치 시 : conda install -c plotly plotly 
from fbprophet.plot import plot_plotly

fig = plot_plotly(m, forecast)
fig.show(renderer="colab")

In [ ]:
# 변동 포인트의 설정
# 빨간 실선은 Trend를 의미(변경점 자동 감지)
from fbprophet.plot import add_changepoints_to_plot
fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)